In [15]:
import pandas as pd
import numpy as np
import logging
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Introduction
this is a tutorial for how to implement an sklearn compatible gaussian naive bayes classifier from scratch. Above, we've imported the libraries we'll need to implement the classifier as well as the built in sklearn GaussianNB classifier we'll use as a reference for the performance of our classifier. If you haven't already, make sure you run the cell above to import the libraries.

# Getting Some Data
We'll use data from the ECE 523 github page for this demo. The datafiles can be found [here](https://github.com/gditzler/UA-ECE-523-Sp2018/tree/master/data). Note that in the function below we use the path to the raw data on github for downloading. 


In [16]:
def load_course_data(dataset_name = "abalone.csv"):
    """
    loads data from the ece_523 github page.
    :param dataset_name: filename of the dataset to load (default is "abalone.csv"
    :return: numpy array containing the csv data
    """
    data_url_format = "https://raw.githubusercontent.com/gditzler/UA-ECE-523-Sp2018/master/data/{0:s}"

    data = pd.read_csv(data_url_format.format(dataset_name))

    return data.values

## Try it Out!
let's just load the default abalone.csv dataset and take a look!

In [17]:
myData = load_course_data()
print(myData.shape)
print(myData)

(4176, 9)
[[-1.15421   -1.44881   -1.43976   ... -1.20508   -1.21284    0.       ]
 [ 0.0537917  0.0500271  0.122116  ... -0.356647  -0.207114   1.       ]
 [-1.15421   -0.699393  -0.432097  ... -0.607527  -0.602222   1.       ]
 ...
 [-1.15421    0.632909   0.676328  ...  0.975296   0.496895   1.       ]
 [ 0.0537917  0.841081   0.777094  ...  0.73354    0.41069    1.       ]
 [-1.15421    1.54887    1.48246   ...  1.78723    1.84026    2.       ]]


So we can see we get a numpy array containing 4176 rows of data in 9 columns. The last column contains the labels. Typically SKLearn Classifiers take the features and labels as separate arguments, so let's write a function to break the data down.

In [18]:
def split_data(data):
    """
    splits a n-by-m numpy array containing n datapoints with m - 1 features, and a last column containing labels
    into data (X) and labels (Y)
    :param data: the numpy array to split up
    :return: 
    """
    Y = data[:, -1]
    X = data[:, 0:data.shape[1] - 1]

    return X, Y

Now let's see how it works...

In [19]:
data, labels = split_data(myData)
# let's split the data into training and testing data
X, X_t, Y, Y_t = train_test_split(
        data,
        labels,
        test_size=.5,
        random_state=42
    ) 
print(data.shape)
print(data)
print(X.shape)
print(X)
print(Y.shape)
print(Y)

(4176, 8)
[[-1.15421   -1.44881   -1.43976   ... -1.17077   -1.20508   -1.21284  ]
 [ 0.0537917  0.0500271  0.122116  ... -0.463444  -0.356647  -0.207114 ]
 [-1.15421   -0.699393  -0.432097  ... -0.64816   -0.607527  -0.602222 ]
 ...
 [-1.15421    0.632909   0.676328  ...  0.74847    0.975296   0.496895 ]
 [ 0.0537917  0.841081   0.777094  ...  0.773248   0.73354    0.41069  ]
 [-1.15421    1.54887    1.48246   ...  2.64068    1.78723    1.84026  ]]
(2088, 8)
[[-1.15421    0.882716   0.978626  ...  1.23053    0.601258   0.942289 ]
 [-1.15421   -2.07333   -2.09474   ... -1.45235   -1.42859   -1.50019  ]
 [ 0.0537917  0.54964    0.777094  ...  0.419586   0.523713   1.30148  ]
 ...
 [ 0.0537917 -0.0332418  0.222882  ... -0.258455  -0.210681  -0.02752  ]
 [ 0.0537917  0.424737   0.52518   ... -0.118792   0.240903   0.428649 ]
 [ 0.0537917  0.591275   0.575563  ...  0.712428   0.57845    0.475344 ]]
(2088,)
[2. 0. 2. ... 2. 1. 2.]


# Getting Started with Naive Bayes
for a Naive Bayes Classifier $P(Y_k | X) = \frac{P(Y_k) \times P(X | Y_k)}{P(X)}$ or the $Posterior = \frac{prior \times liklihood}{evidence}$ now $evidence$ can be hard to find, but fortunately, it's also the least important, since it's the same for all $Y_i$, and therefore scales all of the posterior probablities proportionally. So we can simply leave it out. That leaves us with $P(Y_k | X) \propto P(Y_k) \times P(X | Y_k)$ as the fundamental equation of our naive bayes classifier.

## Calculating the prior probability
to create our classifier, it's easiest to start by calculating the prior probabilities for all of the classes in the dataset. $P(Y_k) = \frac{number\ of\ occurences\ of\ Y_k\ in\ the\ dataset}{total\ size\ of\ the\ data\ set}$

## Calculating the liklihood or class conditional probability
This is where the "Gaussian" and "Naive" from the name of the classifier come into play. First, we're going to assume the class conditional probabilities are normally distributed, second we're going to assume that they're independent from one another. This makes our posterior probability equation $P(Y_k | X) \propto P(Y_k)\prod_{i=1}^{n-features}P(X_i | Y_k)$. For model training purposes this means for each class we're going to want the mean and variance for each feature that is part of a data point in that class.

In [20]:
def fit_model(X,Y):
        # store the number of classes
        n_classes = np.unique(Y).size
        print("n_classes:{0:d}".format(n_classes))

        # store the number of occurences of each class in the training data
        class_count = np.zeros(n_classes)
        for label in range(0,n_classes):
            class_count[label] = Y[Y == label].size
        print(("class_count:" + np.array_str(class_count)))

        # compute the prior probability of each class from the training data
        prior = np.zeros(n_classes)
        for idx, value in enumerate(class_count):
            prior[idx] = value / np.sum(class_count)

        print("prior:" + np.array_str(prior))
        print("sum or prior = {0:f}".format(np.sum(prior)))


        n_features = X.shape[1]
        print("n_features:{0:d}".format(n_features))

        # compute the mean and variance of each feature per class
        mean = np.zeros((n_classes, n_features))
        variance = np.zeros(mean.shape)
        for lbl in range(0, n_classes):
            # group training data by label
            X_lbl = X[Y == lbl, :]
            mean[lbl] = np.mean(X_lbl, axis = 0)
            variance[lbl] = np.var(X_lbl, axis = 0)

        print("mean:\n" + np.array_str(mean))
        print("variance:\n" + np.array_str(variance))

        return n_classes, class_count, prior, n_features, mean, variance

Once again, let's give it a try. I've thrown some print statements into the function so you can see what's going on.

In [21]:
n_classes, class_count, prior, n_features, mean, variance = fit_model(X,Y)

n_classes:3
class_count:[679. 683. 726.]
prior:[0.32519157 0.32710728 0.34770115]
sum or prior = 1.000000
n_features:8
mean:
[[ 0.55727182 -0.86169938 -0.87419106 -0.75990272 -0.81258685 -0.72754601
  -0.79575554 -0.85574846]
 [-0.18851684  0.28544751  0.26728962  0.19620967  0.17310374  0.22689534
   0.1938192   0.12234517]
 [-0.37882634  0.59838586  0.61491132  0.60300419  0.6472      0.52253034
   0.6162865   0.72332738]]
variance:
[[1.00804946 0.85582724 0.81359802 1.46653394 0.3747472  0.4277836
  0.37245287 0.3223442 ]
 [0.86214067 0.51027803 0.49020357 0.55115871 0.63525016 0.75351929
  0.69678389 0.51698034]
 [0.63292535 0.4735234  0.46277384 0.47199598 0.8959942  0.99677558
  0.9276522  0.88113948]]


# Making Predictions
to make predictions with this model, we'll be finding the label with the maximum posterior probability as produced by the equation above. Howver, we'll be making one small tweak. We'll be using the log probabilities. This is useful for preventing floating point underflows, because otherwise we could potentially be multiplying a lot of numbers that are $<1$ together. Note that this makes the equation for the posterior: $log(P(Y_k | x)) \propto log(P(Y_k)) + \sum_{i=1}^{n-features}log(P(X_i | Y_k)$. To start with we'll need a function to compute $\sum log(P(X_i|Y_k)$ from the gaussian distribution.  

In [22]:
def log_liklihood(X_test, n_classes, mean, variance):
    class_log_liklihoods = []
    for idx in np.arange(0, n_classes):
            liklihood = - 0.5 * np.sum(np.log(2 * np.pi * variance[idx,:]))
            liklihood += -0.5 * np.sum(np.square((X - mean[idx, :])) / ( variance[idx,:]) , 1)
            class_log_liklihoods.append(liklihood)

    class_log_liklihoods = np.array(class_log_liklihoods).T
    print("log_liklihood:class_log_liklihood:\n" + np.array_str(class_log_liklihoods))
    return class_log_liklihoods

l_liklihood = log_liklihood(X_t, n_classes, mean, variance)

log_liklihood:class_log_liklihood:
[[-27.55305659  -8.78586156  -7.01193884]
 [-11.17139943 -27.94739505 -35.68392261]
 [-24.85525792  -9.25451132  -7.20786552]
 ...
 [ -9.27791008  -5.95421557  -8.09787932]
 [-13.83868839  -5.70351967  -6.59390769]
 [-18.53769462  -6.17448008  -6.40747287]]


finally, we'll combine these with the posterior to get our prediction probabilities

In [23]:
def predict_log_proba(prior, log_liklihood):
    log_prior = np.log(prior)
    
    log_probas = np.zeros(log_liklihood.shape)
    
    for idx, row in enumerate(log_liklihood):
        log_probas[idx] = np.add(row, log_prior)
        
    print("log_probas\n" + np.array_str(log_probas))
    return log_probas

log_probas = predict_log_proba(prior, l_liklihood)

log_probas
[[-28.67639741  -9.90332865  -8.06835078]
 [-12.29474025 -29.06486214 -36.74033454]
 [-25.97859874 -10.37197841  -8.26427745]
 ...
 [-10.4012509   -7.07168266  -9.15429126]
 [-14.96202921  -6.82098676  -7.65031963]
 [-19.66103544  -7.29194716  -7.4638848 ]]


## Final Prediction:
the last thing to do is select the column corresponding to the label with the maximum posterior probability...

In [24]:
def predict(log_probas):
    predictions = np.zeros(log_probas.shape[0])
    for idx, row in enumerate(log_probas):
        predictions[idx] = np.argmax(row)

    print("predictions: " + np.array_str(predictions))
    return predictions

predict(log_probas)

predictions: [2. 0. 2. ... 1. 1. 1.]


array([2., 0., 2., ..., 1., 1., 1.])

# An SKLearn Classifier Comparison
Below I've wrapped the functions from the examples above into an sklearn compatible classifier, complete with "score" function that can be used in k-fold validation.

In [25]:
from sklearn.base import BaseEstimator, ClassifierMixin

class GaussianNaiveBayesFromScratch(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.n_classes  = None
        self.prior = None
        self.n_features = None
        self.class_count = None
        self.mean = None
        self.variance = None


    def fit(self, X, Y):

        # store the number of classes
        self.n_classes = np.unique(Y).size
        #logging.debug("GaussianNaiveBayesFromScratch:n_classes:{0:d}".format(self.n_classes))

        # store the number of occurences of each class in the training data
        self.class_count = np.zeros(self.n_classes)
        for label in range(0,self.n_classes):
            self.class_count[label] = Y[Y == label].size
        #logging.debug(("GaussianNaiveBayesFromScratch:class_count:" + np.array_str(self.class_count)))

        # compute the prior probability of each class from the training data
        self.prior = np.zeros(self.n_classes)
        for idx, value in enumerate(self.class_count):
            self.prior[idx] = value / np.sum(self.class_count)

        #logging.debug("GaussianNaiveBayesFromScratch:prior:" + np.array_str(self.prior))
        #logging.debug("GaussianNaiveBayesFromScratch:sum or prior = {0:f}".format(np.sum(self.prior)))


        self.n_features = X.shape[1]
        #logging.debug("GaussianNaiveBayesFromScratch:n_features:{0:d}".format(self.n_features))

        # compute the mean and variance of each feature per class
        self.mean = np.zeros((self.n_classes, self.n_features))
        self.variance = np.zeros(self.mean.shape)
        for lbl in np.arange(0, self.n_classes):
            # group training data by label
            X_lbl = X[Y == lbl, :]
            mean = np.mean(X_lbl, axis = 0)
            variance = np.var(X_lbl, axis = 0)

            self.mean[lbl] = mean
            self.variance[lbl] = variance

        #logging.debug("GaussianNaiveBayesFromScratch:mean:\n" + np.array_str(self.mean))
        #logging.debug("GaussianNaiveBayesFromScratch:variance:\n" + np.array_str(self.variance))

        return self


    def _log_liklihood(self, X):

        class_log_liklihoods = []
        for idx in np.arange(0, self.n_classes):
            liklihood = - 0.5 * np.sum(np.log(2 * np.pi * self.variance[idx,:]))
            liklihood += -0.5 * np.sum(np.square((X - self.mean[idx, :])) / ( self.variance[idx,:]) , 1)
            class_log_liklihoods.append(liklihood)

        class_log_liklihoods = np.array(class_log_liklihoods).T
        #logging.debug("GaussianNaiveBayesFromScratch:_log_liklihood:class_log_liklihood:\n" + np.array_str(class_log_liklihoods))
        return class_log_liklihoods

    @property
    def _log_prior(self):
        log_prior = np.log(self.prior)

        #logging.debug("GaussianNaiveBayesFromScratch:_log_prior:log_prior\n" + np.array_str(log_prior))

        return log_prior

    def predict_log_proba(self, X):
        log_liklihood = self._log_liklihood(X)

        log_probas = np.zeros(log_liklihood.shape)

        for idx, row in enumerate(log_liklihood):
            # this is the sum log(p(Y)) + log(p(X | Y))
            log_probas[idx] = np.add(row , self._log_prior)

        #logging.debug("GaussinaNaiveBayesFromScratch:predict_log_probas:log_probas:\n" + np.array_str(log_probas))

        return log_probas

    def predict(self, X):
        prob = self.predict_log_proba(X)

        predictions = np.zeros(prob.shape[0])
        for idx, row in enumerate(prob):
            predictions[idx] = np.argmax(row)


        return predictions

    def score(self, X, Y):
        predicted = self.predict(X)

        correct = predicted[predicted == Y]

        return correct.shape[0] / Y.shape[0]


let's see how this compares to the built in SKLearn Naive Bayes Classifier.

In [26]:
from sklearn.model_selection import cross_val_score

def compare_classifiers(data_set = "abalone.csv"):
   
    data = load_course_data(data_set)
    data, labels = split_data(data)
    
    GNB_from_scratch = GaussianNaiveBayesFromScratch()
    GNB_fs_scores = cross_val_score(GNB_from_scratch, data, labels, cv=5)
    
    print("GaussianNaiveBayesFromScratch 5-fold cross validation score: {0:f}".format(np.mean(GNB_fs_scores)))
    
    sk_GNB = GaussianNB()
    sk_scores = cross_val_score(sk_GNB, data, labels, cv=5)
    
    print("GaussianNB 5-fold cross validation score: {0:f}".format(np.mean(sk_scores)))
    

    

Try out some of the different data-sets on the github page by changing out the filename in the command below

In [27]:
compare_classifiers("adult_test.csv")

GaussianNaiveBayesFromScratch 5-fold cross validation score: 0.809582
GaussianNB 5-fold cross validation score: 0.809582
